# Jervis Chan Jun Yong
Lab Group: TS8


In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

In [ ]:
SFlow_data = pd.read_csv("SFlow_Data_lab4.csv", header = None)
SFlow_data.columns = ["Type", "sflow_agent_address", "inputPort", "outputPort", "src_MAC", "dst_MAC", "ethernet_type", "in_vlan", "out_vlan", "src_IP", "dst_IP", "IP_protocol", "ip_tos", "ip_ttl", "udp_src_port/tcp_src_port/icmp_type", "udp_dst_port/tcp_dst_port/icmp_code", "tcp_flags", "packet_size", "IP_size", "sampling_rate"]
SFlow_data.head()

# Top Talkers and Listeners

In [ ]:
top_5_talkers = SFlow_data.pivot_table(index=['src_IP'], aggfunc='size').sort_values(ascending=False).iloc[0:5]
top_5_talkers

In [ ]:
top_5_listeners = SFlow_data.pivot_table(index=['dst_IP'], aggfunc='size').sort_values(ascending=False).iloc[0:5]
top_5_listeners

# Transport Protocol

In [ ]:
top_5_packets = SFlow_data.pivot_table(index=['IP_protocol'], aggfunc='size').sort_values(ascending=False).iloc[0:5]
total_packets = top_5_packets.sum()
print(top_5_packets)

In [ ]:
top_5_packets /= total_packets
top_5_packets

# Applications Protocol

In [ ]:
top_5_packets = SFlow_data.pivot_table(index=['udp_dst_port/tcp_dst_port/icmp_code'], aggfunc='size').sort_values(ascending=False).iloc[0:5]
top_5_packets

# Traffic

In [ ]:
sampling_rates = SFlow_data.pivot_table(index=['sampling_rate'], aggfunc='size').sort_values(ascending=False)
sampling_rates

In [ ]:
# Base10 MB to B Conversion
traffic = SFlow_data["IP_size"] * 1000
total_traffic = traffic.sum() / 10**6
print("Total Traffic (1): ", total_traffic, "MB")

# Base2 MB to B Conversion
traffic = SFlow_data["IP_size"] * 1000
total_traffic = traffic.sum() / 2**20
print("Total Traffic (2): ", total_traffic, "MB")

# Additional Analysis

## Top 5 Communication Pairs

In [ ]:
pair_data = SFlow_data
pair_data['pair_IP'] = None

for index, row in pair_data.iterrows():
    pair = [row['src_IP'], row['dst_IP']]
    pair.sort()
    pair_ = tuple(pair)
    pair_data.at[index, 'pair_IP'] = pair_

pair_data_grouped = pair_data.groupby('pair_IP').size().reset_index(name='Count')
pair_data_grouped = pair_data_grouped.sort_values(['Count'], ascending=False)
top_5_pairs = pair_data_grouped.head(5)
top_5_pairs

## Visualisations of Top Listeners

In [ ]:
for listener in top_5_listeners.keys():
    listener_network = SFlow_data.loc[SFlow_data['dst_IP'] == listener]
    listener_network = listener_network[['src_IP', 'dst_IP']]
    print("dst_IP:", listener)

    G = nx.Graph()
    G = nx.from_pandas_edgelist(listener_network, 'src_IP', 'dst_IP')
    print(list(G.nodes))
    nx.draw(G, with_labels=False)
    plt.show()

## Visualisations of Top Talkers

In [ ]:
for talker in top_5_talkers.keys():
    talker_network = SFlow_data.loc[SFlow_data['src_IP'] == talker]
    talker_network = talker_network[['src_IP', 'dst_IP']]
    print("src_IP:", talker)

    G = nx.Graph()
    G = nx.from_pandas_edgelist(talker_network, 'src_IP', 'dst_IP')
    print(list(G.nodes))
    nx.draw(G, with_labels=False)
    plt.show()